In [2]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order_condition import Create, OrderCondition
from ibapi.order import *
#import ib_insync
#from ib_insync import *
import datetime as dt
import time
import ibapi
import pandas as pd
import threading

In [29]:

class IBapi(EWrapper, EClient):
    def __init__(self):
	    EClient.__init__(self, self)
	    self.data = [] #Initialize variable to store candle
    
    def tickPrice(self, reqId, tickType, price, attrib):
	    if tickType == 2 and reqId == 1:
		    print('The current ask price is: ', price)
    
    def nextValidId(self, orderId: int):
	    super().nextValidId(orderId)
	    self.nextorderId = orderId
	    print('The next valid order id is: ', self.nextorderId)

    def orderStatus(self, orderId, status, filled, remaining, avgFullPrice, permId, parentId, lastFillPrice, clientId, whyHeld, mktCapPrice):
	    print('orderStatus - orderid:', orderId, 'status:', status, 'filled', filled, 'remaining', remaining, 'lastFillPrice', lastFillPrice)
	
    def openOrder(self, orderId, contract, order, orderState):
	    print('openOrder id:', orderId, contract.symbol, contract.secType, '@', contract.exchange, ':', order.action, order.orderType, order.totalQuantity, orderState.status)

    def execDetails(self, reqId, contract, execution):
	    print('Order Executed: ', reqId, contract.symbol, contract.secType, contract.currency, execution.execId, execution.orderId, execution.shares, execution.lastLiquidity)
    
    def error(self, reqId, errorCode, errorString):
        if errorCode == 202:
            print('order canceled')


def run_loop():
    app.run()

#Function to create FX Order contract
def FX_order(symbol):
    contract = Contract()
    contract.symbol = symbol[:3]
    contract.secType = 'CASH'
    contract.exchange = 'IDEALPRO'
    contract.currency = symbol[3:]
    return contract

def Stock_contract(symbol, secType='STK', exchange='SMART', currency='USD'):
	''' custom function to create stock contract '''
	contract = Contract()
	contract.symbol = symbol
	contract.secType = secType
	contract.exchange = exchange
	contract.currency = currency
	return contract

def contractDetails(self, reqId: int, contractDetails):
	    self.contract_details[reqId] = contractDetails

def get_contract_details(self, reqId, contract):
    self.contract_details[reqId] = None
    self.reqContractDetails(reqId, contract)
    #Error checking loop - breaks from loop once contract details are obtained
    for err_check in range(50):
        if not self.contract_details[reqId]:
            time.sleep(0.1)
        else:
            break
    #Raise if error checking loop count maxed out (contract details not obtained)
    if err_check == 49:
        raise Exception('error getting contract details')
    #Return contract details otherwise
    return app.contract_details[reqId].contract 


app = IBapi()


# ReqMktData

1) The ReqId is a unique positive integer you assign to your request which will be included in the response. This way, if you make several market data requests at the same time, you’ll know which returned data belongs to which asset.

2) This is the contract object

3) The tickType, left empty in this example, allows you to specify what kind of data you’re looking for. Since the ask price is part of the default dataset returned, we don’t need to specify a tickType. You can run the code snippet below to get a full list of all the tickTypes available.

4) This is if you want snapshot data for an asset that you do not have a subscription to. If you have a market data subscription, or one is not required, set this to False.

5) The fifth item is to obtain a snapshot rather than streaming data. This is for assets you already have a subscription for, or if a subscription is not required.

6) Not used by anyone. For internal use of IB, if they only know it exists...


### TYPE DE DATA EXISTANTES MISE A PART LES CLASSIQUES CLOSE, BID, ASK...
from ibapi.ticktype import TickTypeEnum

for i in range(91):
	print(TickTypeEnum.to_str(i), i)

####### i.e the numerical value for the ask price is 2, hence the if statement in the tickPrice function in our script to filter out only the ask price

In [3]:
app.connect(host='127.0.0.1', port=7496, clientId=72)

time.sleep(1)
if app.isConnected() == True:
    print('Connexion établie')
else:
    print('Problème de connexion')

#Start the socket in a thread
api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()

time.sleep(1) #Sleep interval to allow time for connection to server

#Create contract object
apple_contract = Contract()
apple_contract.symbol = 'AAPL'
apple_contract.secType = 'STK'
apple_contract.exchange = 'SMART'
apple_contract.currency = 'USD'

#Request Market Data
app.reqMktData(1, apple_contract, '', False, False, [])

time.sleep(10) #Sleep interval to allow time for incoming price data
app.disconnect()

if app.isConnected() == True:
    print('Connexion établie')
else:
    print('Deconnexion effectuée')

Connexion établie
The next valid order id is:  1
The current ask price is:  467.8
The current ask price is:  467.81
The current ask price is:  467.83
The current ask price is:  467.85
The current ask price is:  467.84
The current ask price is:  467.85
The current ask price is:  467.84
The current ask price is:  467.87
The current ask price is:  467.86
The current ask price is:  467.87
The current ask price is:  467.82
The current ask price is:  467.84
The current ask price is:  467.86
The current ask price is:  467.87
The current ask price is:  467.91
Deconnexion effectuée


# reqHistoricalData

### 1) The ReqId is a unique positive integer you assign to your request which will be included in the response. This way, if you make several market data requests at the same time, you’ll know which returned data belongs to which asset.

### 2) This is the contract object

### 3) Interval, for example ‘2 D’ (str, respect the space) : it stands for two days. The interval is calculated from the prior day’s close so if you chose ‘1 D’ , depending on the time of day, you might get less than 10 candles. 

S	Seconds

D	Day

W	Week

M	Month

Y	Year

### 4) Time Period is straightforward and we set this to ‘1 hour’ as we are looking for hourly candles.

1 secs	5 secs	10 secs	15 secs	30 secs

1 min	2 mins	3 mins	5 mins	10 mins	15 mins	20 mins	30 mins

1 hour	2 hours	3 hours	4 hours	8 hours

1 day

1 week

1 month

### 5) The Data Type will typically be either BID, ASK, or MIDPOINT. On most charting platforms, the BID price is used.


### 6) RTH stands for Regular Trading Hours. It’s mostly used for stocks and if you’re looking for pre-market data, set this to 1.

### 7) Time Format. Set it to 1 if you want the response data to contain readable time and set it to 2 for Epcoh (Unix) time. The second option makes it much easier to convert to a Python DateTime object.

### 8) Streaming : if it is set to True, it will keep updating price bars every five seconds (even if the candle has not closed).

Note: Since the last candle sent over by IB has likely not closed, it is a good idea to verify whether it has or not, and discard the last candle if needed to ensure accurate data





In [4]:
app.connect('127.0.0.1', 7496, 12)
app.run()
time.sleep(1)
if app.isConnected() == True:
    print('Connexion établie')
else:
    print('Problème de connexion')

#Start the socket in a thread
#api_thread = threading.Thread(target=run_loop, daemon=True)
#api_thread.start()

time.sleep(1) #Sleep interval to allow time for connection to server

#Create contract object
eurusd_contract = Contract()
eurusd_contract.symbol = 'EUR'
eurusd_contract.secType = 'CASH'
eurusd_contract.exchange = 'IDEALPRO'
eurusd_contract.currency = 'USD'

#Create contract object
apple_contract = Contract()
apple_contract.symbol = 'AAPL'
apple_contract.secType = 'STK'
apple_contract.exchange = 'SMART'
apple_contract.currency = 'USD'

#Request historical candles
#app.reqHistoricalData(1, apple_contract, '', '2 D', '1 hour', 'BID', 0, 2, False, [])
app.reqHistoricalData(9, apple_contract,'20200819 12:00:00' ,'6 M', '1 day', 'MIDPOINT', 0, 2, False, [])
time.sleep(5) #sleep to allow enough time for data to be returned

df = pd.DataFrame(app.data, columns=['DateTime','Close'])
df['DateTime'] = pd.to_datetime(df['DateTime'],unit='s') 
#df.to_csv('EURUSD_Hourly.csv')  

print(df)


app.disconnect()

if app.isConnected() == True:
    print('Connexion établie')
else:
    print('Deconnexion effectuée')

setting nextValidOrderId: %d 1
Error. Id:  -1  Code:  2104  Msg:  La connexion de données de marché est OK:hfarm
Error. Id:  -1  Code:  2104  Msg:  La connexion de données de marché est OK:usfarm.nj
Error. Id:  -1  Code:  2104  Msg:  La connexion de données de marché est OK:jfarm
Error. Id:  -1  Code:  2104  Msg:  La connexion de données de marché est OK:usfuture
Error. Id:  -1  Code:  2104  Msg:  La connexion de données de marché est OK:eufarm
Error. Id:  -1  Code:  2104  Msg:  La connexion de données de marché est OK:cashfarm
Error. Id:  -1  Code:  2104  Msg:  La connexion de données de marché est OK:usfarm
Error. Id:  -1  Code:  2106  Msg:  La connexion de données HMDS est OK:euhmds
Error. Id:  -1  Code:  2106  Msg:  La connexion de données HMDS est OK:cashhmds
Error. Id:  -1  Code:  2106  Msg:  La connexion de données HMDS est OK:fundfarm
Error. Id:  -1  Code:  2106  Msg:  La connexion de données HMDS est OK:ushmds
Error. Id:  -1  Code:  2158  Msg:  La connexion de la batterie de s

AttributeError: 'EClient' object has no attribute 'data'

In [25]:
app.connect('127.0.0.1', 7496, 72)

time.sleep(1)

#Start the socket in a thread
api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()

time.sleep(1) #Sleep interval to allow time for connection to server
if app.isConnected() == True:
    print('Connexion établie')
else:
    print('Problème de connexion')


The next valid order id is:  1
Connexion établie


In [26]:
queryTime = (dt.datetime.today() - dt.timedelta(days=180)).strftime("%Y%m%d %H:%M:%S")
app.reqHistoricalData(9, apple_contract,queryTime,'6 M', '1 day', 'MIDPOINT', 0, 2, False, [])

In [23]:
app.data

[]

In [24]:
app.disconnect()

if app.isConnected() == True:
    print('Connexion établie')
else:
    print('Deconnexion effectuée')

Deconnexion effectuée


# app.placeOrder

1)

2)

3)

4)



In [5]:
app = IBapi()
app.connect('127.0.0.1', 7497, 123)
time.sleep(1)
if app.isConnected() == True:
    print('Connexion établie')
else:
    print('Problème de connexion')

app.nextorderId = None

#Start the socket in a thread
api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()

#Create order object
order = Order()
order.action = 'BUY'
order.totalQuantity = 100000
order.orderType = 'LMT'
order.lmtPrice = '1.10'

#Place order
app.placeOrder(app.nextorderId, FX_order('EURUSD'), order)
#app.nextorderId += 1

time.sleep(3)

#Cancel order 
print('cancelling order')
app.cancelOrder(app.nextorderId)

time.sleep(3)
app.disconnect()

if app.isConnected() == True:
    print('Connexion établie')
else:
    print('Deconnexion effectuée')


Connexion établie
The next valid order id is:  11
cancelling order
Deconnexion effectuée


# Bracket Orders

Placing Take profit or stop loss

In [6]:
app.connect('127.0.0.1', 7497, 123)

time.sleep(1)

if app.isConnected() == True:
    print('Connexion établie')
else:
    print('Problème de connexion')

app.nextorderId = None

#Start the socket in a thread
api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()

#Check if the API is connected via orderid
#while True:
#	if isinstance(app.nextorderId, int):
#		print('connected')
#		print()
#		break
#	else:
#		print('waiting for connection')
#		time.sleep(1)

#Create order object
order = Order()
order.action = 'BUY'
order.totalQuantity = 100000
order.orderType = 'LMT'
order.lmtPrice = '1.10'
order.orderId = app.nextorderId
app.nextorderId += 1
order.transmit = False

#Create stop loss order object
stop_order = Order()
stop_order.action = 'SELL'
stop_order.totalQuantity = 100000
stop_order.orderType = 'STP'
stop_order.auxPrice = '1.09'
stop_order.orderId = app.nextorderId
app.nextorderId += 1
stop_order.parentId = order.orderId
order.transmit = True

#Place orders
app.placeOrder(order.orderId, FX_order('EURUSD'), order)
app.placeOrder(stop_order.orderId, FX_order('EURUSD'), stop_order)

time.sleep(3)

#Cancel order 
print('cancelling order')
app.cancelOrder(order.orderId)

time.sleep(3)

app.disconnect()

if app.isConnected() == True:
    print('Connexion établie')
else:
    print('Deconnexion effectuée')


The next valid order id is:  12
Connexion établie
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting

# ibapi.order_condition :: ORDER CONDITIONS:

## PLACE AN ORDER CROSS CONDITION ON ANOTHER

Let's say we can fire an order for Apple when Google hits a certain price

Since we are using a special order feature, we need to import two classes from ibapi.order_condition.

from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order_condition import Create, OrderCondition
from ibapi.order import *

import threading
import time
Along with that, we have some of the same imports used in prior examples to create a contract and an order object.

class IBapi(EWrapper, EClient):
	def __init__(self):
		EClient.__init__(self, self)
		self.contract_details = {} #Contract details will be stored here using reqId as a dictionary key

	def nextValidId(self, orderId: int):
		super().nextValidId(orderId)
		self.nextorderId = orderId
		print('The next valid order id is: ', self.nextorderId)

	def orderStatus(self, orderId, status, filled, remaining, avgFullPrice, permId, parentId, lastFillPrice, clientId, whyHeld, mktCapPrice):
		print('orderStatus - orderid:', orderId, 'status:', status, 'filled', filled, 'remaining', remaining, 'lastFillPrice', lastFillPrice)
	
	def openOrder(self, orderId, contract, order, orderState):
		print('openOrder id:', orderId, contract.symbol, contract.secType, '@', contract.exchange, ':', order.action, order.orderType, order.totalQuantity, orderState.status)

	def execDetails(self, reqId, contract, execution):
		print('Order Executed: ', reqId, contract.symbol, contract.secType, contract.currency, execution.execId, execution.orderId, execution.shares, execution.lastLiquidity)
Next, we’ve overwritten a few more functions that will return data once the order has been sent, and when the order has been executed. All we are doing is directing the API to print this information out to the console, just to illustrate how they work.

The next code snippet is a bit more pertinent to what we are trying to accomplish. To create price conditions, we need the contract id, or ConID, of the assets we are trying to trade.

You can get this id by searching the IB Contract and Symbol Database. But a much easier way is to use the reqContractDetails functions of the API. It will return a contract with the ConID already filled in.

	def contractDetails(self, reqId: int, contractDetails):
		self.contract_details[reqId] = contractDetails

	def get_contract_details(self, reqId, contract):
		self.contract_details[reqId] = None
		self.reqContractDetails(reqId, contract)
		#Error checking loop - breaks from loop once contract details are obtained
		for err_check in range(50):
			if not self.contract_details[reqId]:
				time.sleep(0.1)
			else:
				break
		#Raise if error checking loop count maxed out (contract details not obtained)
		if err_check == 49:
			raise Exception('error getting contract details')
		#Return contract details otherwise
		return app.contract_details[reqId].contract
There are two functions to get the updated contract that includes a ConID. The first is contractDetails which is a function of the EWrapper. When we request contract details, it will get returned here.

We will store whatever is returned here in a dictionary file. The request id, or reqId, that we use to make the request, will be used as the key value for the dictionary.

Next, we have created a custom function for requesting contract details. To access it, we have to pass through a reqId and the contract that we are requesting details for.

This function will send the request to the API, and then wait for a response to be returned.

We want to do some error checking at this point. So a loop has been set to run 50 times. It checks to see if our contract details have been returned, and if so, the loop is broken.

If the loop runs a full 50 times, meaning it didn’t successfully break out, the value of err_check will be 49. In this case, we will raise an exception to alert us that there is a problem getting the contract details.

Now that we’ve finished our class functions, let’s move on to the main script. Here we’ve created two functions.

def run_loop():
	app.run()

def Stock_contract(symbol, secType='STK', exchange='SMART', currency='USD'):
	''' custom function to create stock contract '''
	contract = Contract()
	contract.symbol = symbol
	contract.secType = secType
	contract.exchange = exchange
	contract.currency = currency
	return contract
The first is simply a function that we will later call to run our app in a thread, similar to prior examples. Anything that needs to be declared, or run when this thread starts, can be added to the run_loop function.

The second function is to simplify creating contracts. We’ve passed in some default values as most stocks will fall into the same category.

app = IBapi()
app.connect('127.0.0.1', 7496, 123)

app.nextorderId = None

#Start the socket in a thread
api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()

#Check if the API is connected via orderid
while True:
	if isinstance(app.nextorderId, int):
		print('connected')
		break
	else:
		print('waiting for connection')
		time.sleep(1)
The above code is similar to the prior examples. We’ve connected to the API, started a thread, and checked to see if the nextorderid exists to confirm a connection.

#Create contracts
apple_contract = Stock_contract('AAPL')
google_contract = Stock_contract('GOOG')
Our next step is to create two contracts, one for GOOG and one for AAPL. Since we are only using the price condition function based on the price of GOOG, the ConID for just that contract is needed.

#Update contract ID
google_contract = app.get_contract_details(101, google_contract)
Therefore, we used our custom get_contract_details function to update the Google contract and not the Apple contract.

We are finally ready to create our price condition. The first step is to create an order condition object.

priceCondition = Create(OrderCondition.Price)
Let’s break down the above code. priceCondition is simply the name of the variable that will store our conditions. You can name this anything you want.

Create is a function from the order_condition.py file found within the API. True to its name, it is used to create an object, or rather, instantiate the right class for our needs.

In this case, we need the PriceCondition class, so that’s where OrderCondition.price comes in.

There are several other types of conditions that you can create and this is where you declare which one you are after.

There are six different types of order conditions in total – Price, Time, Margin, Execution, Volume, and PercentChange.

So for example, if you want to create a condition based on the percentage change for the day, you would use priceCondition = Create(OrderCondition.PercentChange) instead.

Next, we pass through the contract ID of the asset we are setting the condition on, and the exchange it trades from.

priceCondition.conId = google_contract.conId
priceCondition.exchange = google_contract.exchange
This info is already within the contract object, so we just point it to the appropriate attribute of the contract.

We’re ready to set some conditions.

#create conditions
priceCondition.isMore = True
priceCondition.triggerMethod = priceCondition.TriggerMethodEnum.Last
priceCondition.price = 1400.00
We want Google’s price to be above $1400 to execute this trade. So we’ve set the .isMore attribute to True, and have added in a float value of 1400.00 to the .price attribute.

The trigger method that we want to use is the last price that GOOG traded at.

The API requires the trigger method to be entered as an integer, but there is a function called TriggerMethodEnum that will convert the value Last into an integer, which is what we’ve done here.

Our price condition is complete and ready to go. All that’s left now is to add the condition to an order and submit it.

#Create order object
order = Order()
order.action = 'BUY'
order.totalQuantity = 100
order.orderType = 'MKT'
#order.lmtPrice = '300' - optional - you can add a buy limit here
In the above code, we’ve created an order in the same way we’ve done in prior examples. Note that we can create a limit order here. Let’s say we set a limit of $300.

In that scenario, the order would get triggered once GOOG crosses above $1400, but the order would be sent to buy AAPL at $300.

That buy order would remain active no matter what GOOG does next, but won’t be triggered unless AAPL falls back down to $300. If AAPL is already trading at $300 or below at that time, it will get triggered right away.

We are going with a market order, but if you do decide on a limit order, make sure to change the orderType to ‘LMT’.

The price condition we created before still needs to be added to the order. Here’s how to do that:

order.conditions.append(priceCondition)
And don’t forget to set the order.transmit to True.

order.transmit = True
The final step is to submit the order.

app.placeOrder(app.nextorderId, apple_contract, order)
At this point, the order is sitting on IB’s server and it will be managed from there. Even if we shut down our script, that order will remain active and IB will execute it when the conditions are met.





In [1]:
app.connect('127.0.0.1', 7497, 123)

time.sleep(1)

if app.isConnected() == True:
    print('Connexion établie')
else:
    print('Problème de connexion')

app.nextorderId = None

#Start the socket in a thread
api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()

#Check if the API is connected via orderid
while True:
	if isinstance(app.nextorderId, int):
		print('connected')
		break
	else:
		print('waiting for connection')
		time.sleep(1)

#Create contracts
apple_contract = Stock_contract('AAPL')
google_contract = Stock_contract('GOOG')


#Update contract ID
google_contract = app.get_contract_details(101, google_contract)

priceCondition = Create(OrderCondition.Price)

priceCondition.conId = google_contract.conId
priceCondition.exchange = google_contract.exchange

#create conditions
priceCondition.isMore = True
priceCondition.triggerMethod = priceCondition.TriggerMethodEnum.Last
priceCondition.price = 1400.00

#Create order object
order = Order()
order.action = 'BUY'
order.totalQuantity = 100
order.orderType = 'MKT'
#order.lmtPrice = '300' - optional - you can add a buy limit here

order.conditions.append(priceCondition)

order.transmit = True

app.placeOrder(app.nextorderId, apple_contract, order)


if app.isConnected() == True:
    print('Connexion établie')
else:
    print('Deconnexion effectuée')


NameError: name 'app' is not defined

False

In [1]:
import time
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.common import BarData
import pandas as pd
import datetime as dt
from ibapi.order import *
from colorama import Fore, Back, Style
import os

class MyWrapper(EWrapper):

    def nextValidId(self, orderId:int):
        #4 first message received is this one
        self.nextValidOrderId = orderId
        #5 start requests here
        self.start()

    def historicalData(self, reqId:int, bar: BarData):
        #7 data is received for every bar
        DATA.append([bar.date, bar.open, bar.high, bar.low, bar.close, bar.volume])
        return(DATA)

    def historicalDataEnd(self, reqId: int, start: str, end: str):
        #8 data is finished
        #9 this is the logical end of your program
        app.disconnect()        

    def error(self, reqId, errorCode, errorString):
        # these messages can come anytime.
        print("Error. Id: " , reqId, " Code: " , errorCode , " Msg: " , errorString)

    def start(self):
        #6 request data, using fx since I don't have Japanese data
        try:
            app.reqHistoricalData(_id, _contract, '',_duration, _timeframe, "MIDPOINT", 1, 1, False, [])
        except:
            pass
        

def stock_contract(symbol, secType='STK', exchange='SMART', currency='USD'):
    contract = Contract()
    contract.symbol = symbol
    contract.secType = secType
    contract.exchange = exchange
    contract.currency = currency
    contract.exchange = "ISLAND"
    return (contract)

def scrap_base(DATA):
    df = pd.DataFrame(columns=['Date','Open','High','Low','Close','Volume'])
    DATE = []
    OPEN = []
    HIGH = []
    LOW = []
    CLOSE = []
    VOLUME = []
    for x in range(len(DATA)):
        DATE.append(DATA[x][0])
        OPEN.append(DATA[x][1])
        HIGH.append(DATA[x][2])
        LOW.append(DATA[x][3])
        CLOSE.append(DATA[x][4])
        VOLUME.append(DATA[x][5])
    df['Date'] = DATE
    df['Open'] = OPEN
    df['High'] = HIGH
    df['Low'] = LOW
    df['Close'] = CLOSE
    df['Volume'] = VOLUME
    df = pd.DataFrame(columns=['Date','Open','High','Low','Close','Volume'])
    DATE = []
    OPEN = []
    HIGH = []
    LOW = []
    CLOSE = []
    VOLUME = []
    for x in range(len(DATA)):
        DATE.append(str(DATA[x][0][:4]+'-'+DATA[x][0][4:6]+'-'+DATA[x][0][6:8]+' '+DATA[x][0][10:18]))
        OPEN.append(DATA[x][1])
        HIGH.append(DATA[x][2])
        LOW.append(DATA[x][3])
        CLOSE.append(DATA[x][4])
        VOLUME.append(DATA[x][5])
    df['Date'] = DATE
    df['Open'] = OPEN
    df['High'] = HIGH
    df['Low'] = LOW
    df['Close'] = CLOSE
    df['Volume'] = VOLUME
    if _timeframe == '1 D' :
        df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
    elif _timeframe == '1 hour':
        df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d %H:%M:%S')
    df.to_csv(_name+_ticker+'.csv')
    

In [3]:
%%time
SECTOR = []
TICKERS = []
BLACKLIST = ['BRK.B','BF.B']
df_constituents = pd.read_csv('constituents.csv')
df_constituents = df_constituents.drop_duplicates(['Symbol'])
try:
    df_constituents = df_constituents.drop(['Unnamed: 0'],axis=1)
except:
    pass
SECTOR = list(sorted(set(df_constituents.Sector)))        
TICKERS = list(sorted(set(df_constituents.Symbol.to_list())))
for x in BLACKLIST:
    TICKERS.remove(x)
_duration = '5 Y'
_timeframe = '1 hour'
_name = 'BASE_HOURLY/'
try:
    os.mkdir(_name)
except:
    pass
_id = 0

for _ticker in TICKERS:
    print(Fore.BLUE,'Scraping du ticker (hourly)',Fore.YELLOW,_ticker,Style.RESET_ALL)
    _id += 1
    DATA = []
    _contract = stock_contract(_ticker)
    app = EClient(MyWrapper())
    if app.isConnected() == False:
        app.connect("127.0.0.1", 7496, clientId=20)
    else:
        print('deja connecté')
    app.run()
    
    scrap_base(DATA)
    
if app.isConnected() == True:
    print('Deconnexion en cours...')
    app.disconnect()
time.sleep(1)

if app.isConnected() == False:
    print('Deconnexion effectuée')

 Scraping du ticker (hourly)  A 
Error. Id:  -1  Code:  2104  Msg:  La connexion de données de marché est OK:usfarm.nj
Error. Id:  -1  Code:  2104  Msg:  La connexion de données de marché est OK:eufarm
Error. Id:  -1  Code:  2104  Msg:  La connexion de données de marché est OK:usfarm
Error. Id:  -1  Code:  2106  Msg:  La connexion de données HMDS est OK:euhmds
Error. Id:  -1  Code:  2106  Msg:  La connexion de données HMDS est OK:ushmds.nj
Error. Id:  -1  Code:  2106  Msg:  La connexion de données HMDS est OK:fundfarm
Error. Id:  -1  Code:  2106  Msg:  La connexion de données HMDS est OK:ushmds
Error. Id:  -1  Code:  2158  Msg:  La connexion de la batterie de serveurs Sec-def est OK:secdefnj
 Scraping du ticker (hourly)  AAL 
Error. Id:  -1  Code:  2104  Msg:  La connexion de données de marché est OK:usfarm.nj
Error. Id:  -1  Code:  2104  Msg:  La connexion de données de marché est OK:eufarm
Error. Id:  -1  Code:  2104  Msg:  La connexion de données de marché est OK:usfarm
Error. Id:  

In [15]:
x=1
str(DATA[x][0][:4]+'-'+DATA[x][0][4:6]+'-'+DATA[x][0][6:8]+' '+DATA[x][0][10:18])

'2015-08-24 16:00:00'

In [16]:
str(DATA[x][0][:4]+'-'+DATA[x][0][4:6]+'-'+DATA[x][0][6:8]+' '+DATA[x][0][10:18])

'2015-08-24 16:00:00'